In [3]:
from datasets import load_dataset
coNLL = load_dataset("conllpp")
coNLL

Reusing dataset conllpp (/work/wzkariampuzha/.cache/huggingface/datasets/conllpp/conllpp/1.0.0/04f15f257dff3fe0fb36e049b73d51ecdf382698682f5e590b7fb13898206ba2)


DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [4]:
coNLL["train"].features

{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'pos_tags': Sequence(feature=ClassLabel(num_classes=47, names=['"', "''", '#', '$', '(', ')', ',', '.', ':', '``', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'NN|SYM', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB'], names_file=None, id=None), length=-1, id=None),
 'chunk_tags': Sequence(feature=ClassLabel(num_classes=23, names=['O', 'B-ADJP', 'I-ADJP', 'B-ADVP', 'I-ADVP', 'B-CONJP', 'I-CONJP', 'B-INTJ', 'I-INTJ', 'B-LST', 'I-LST', 'B-NP', 'I-NP', 'B-PP', 'I-PP', 'B-PRT', 'I-PRT', 'B-SBAR', 'I-SBAR', 'B-UCP', 'I-UCP', 'B-VP', 'I-VP'], names_file=None, id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(num_classes=9, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], 

In [5]:
coNLL["train"][:2]

{'id': ['0', '1'],
 'tokens': [['EU',
   'rejects',
   'German',
   'call',
   'to',
   'boycott',
   'British',
   'lamb',
   '.'],
  ['Peter', 'Blackburn']],
 'pos_tags': [[22, 42, 16, 21, 35, 37, 16, 21, 7], [22, 22]],
 'chunk_tags': [[11, 21, 11, 12, 21, 22, 11, 12, 0], [11, 12]],
 'ner_tags': [[3, 0, 7, 0, 0, 0, 7, 0, 0], [1, 2]]}

In [17]:
from transformers import BertTokenizer, BertForTokenClassification
import torch
tokenizer = BertTokenizer.from_pretrained("dmis-lab/biobert-large-cased-v1.1")
#tokenizer.tokenize("Aim: There are more than 50 inherited lysosomal storage diseases (LSDs), and this study examined the incidence of clinically diagnosed LSDs in Sweden. Methods: The number of patients diagnosed during 1980-2009 was compiled from the registries of the two Swedish diagnostic laboratories that cover the whole country. Results: We identified 433 patients during the 30-year period, with a total incidence of one in every 6100 births and identified fairly constant annual diagnoses during the last 20 years. Krabbe disease was the most common (one in 39 000) followed by Gaucher disease (one in 47 000), metachromatic leukodystrophy and Salla disease. Gaucher disease was more frequent in Sweden than other European countries, due to a founder effect of the mutation (p.L444P) in northern Sweden. Metachromatic leukodystrophy was one of the most common LSDs, in common with other countries. Salla disease, which is very rare elsewhere, was the fourth most common, stemming from a founder mutation in the Salla region of northern Finland brought to Sweden by immigration. Conclusion: The collective incidence of LSDs in Sweden was essentially equal to other European countries, but with a somewhat different disease pattern. Our findings have implications for diagnostic algorithms and treatment strategies.")
model = BertForTokenClassification.from_pretrained("dmis-lab/biobert-large-cased-v1.1")
#num_labels = 5
#model = BertForTokenClassification.from_pretrained("dmis-lab/biobert-large-cased-v1.1", num_labels = num_labels)

Some weights of the model checkpoint at dmis-lab/biobert-large-cased-v1.1 were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not init

In [7]:
def tokenize_function(example):
    return tokenizer([example["tokens"],example["ner_tags"]], padding = 'max_length', truncation=True, max_length=128, is_split_into_words=True)
    #return tokenizer([example["tokens"],example["ner_tags"]], is_split_into_words=True)

    #return tokenizer(example["id"],example["tokens"],example["pos_tags"],example["chunk_tags"],
    #                 example["ner_tags"], padding="max_length",truncation=True,max_length=128)

In [8]:
tokenized_data = coNLL.map(tokenize_function)
#tokenized_data = coNLL.map(tokenize_function,batched=True)
print(tokenized_data.column_names)

  0%|          | 0/14041 [00:00<?, ?ex/s]

  0%|          | 0/3250 [00:00<?, ?ex/s]

  0%|          | 0/3453 [00:00<?, ?ex/s]

{'train': ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask'], 'validation': ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask'], 'test': ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask']}


In [24]:
#From the first data preprocessing video
tokenized_data["train"].features["tokens"]

{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'pos_tags': Sequence(feature=ClassLabel(num_classes=47, names=['"', "''", '#', '$', '(', ')', ',', '.', ':', '``', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'NN|SYM', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB'], names_file=None, id=None), length=-1, id=None),
 'chunk_tags': Sequence(feature=ClassLabel(num_classes=23, names=['O', 'B-ADJP', 'I-ADJP', 'B-ADVP', 'I-ADVP', 'B-CONJP', 'I-CONJP', 'B-INTJ', 'I-INTJ', 'B-LST', 'I-LST', 'B-NP', 'I-NP', 'B-PP', 'I-PP', 'B-PRT', 'I-PRT', 'B-SBAR', 'I-SBAR', 'B-UCP', 'I-UCP', 'B-VP', 'I-VP'], names_file=None, id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(num_classes=9, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], 

In [10]:
type(tokenized_data["train"])

datasets.arrow_dataset.Dataset

In [11]:
print(tokenized_data["train"].shape)

(14041, 8)


In [65]:
type(tokenized_data["train"][:2])

dict

In [21]:
from transformers import DataCollatorWithPadding, DataCollatorForTokenClassification
data_collator_example = DataCollatorWithPadding(tokenizer)
data_collator_token = DataCollatorForTokenClassification(tokenizer)

In [19]:
from transformers import TrainingArguments
training_args = TrainingArguments("test-trainer")

In [72]:
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
    def __len__(self):
        return self.features.shape[0]
    def __getitem__(self, idx):
        item = self.features[idx]
        label = self.labels[idx]
        return item, label

In [73]:
train_data = CustomDataset(tokenized_data["train"],tokenized_data["train"][:])
#eval_data = CustomDataset(tokenized_data["validation"])

In [74]:
train_data.features.shape[0]

14041

In [75]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(eval_data, batch_size=64)

In [76]:
from transformers import Trainer
trainer = Trainer(model,
                  training_args,
                  train_dataset=train_data,
                  eval_dataset=eval_data,
                 data_collator=data_collator_example,
                 tokenizer=tokenizer)
trainer.train()

***** Running training *****
  Num examples = 14041
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 5268
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


KeyError: 10813

In [ ]:
# Need to create a custom class that implements dataset abstract class from pytorch
#https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#creating-a-custom-dataset-for-your-files

In [ ]:
#From the first data preprocessing video
#tokenized_data = tokenized_data.remove_columns(["id", "pos_tags","chunk_tags"])
#tokenized_data = tokenized_data.with_format("torch")
#tokenized_data["train"]
#small_train_dataset = tokenized_data["train"].select(range(100))

In [ ]:
#Start the Trainer
from transformers import Trainer
trainer = Trainer(model,
                  training_args,
                  train_dataset=tokenized_data["train"],
                  eval_dataset=tokenized_data["validation"],
                 data_collator = data_collator_example,
                 tokenizer=tokenizer)
trainer.train()

In [ ]:
predictions = trainer.predict(tokenized_data["validation"])
#Find label_ids
print(predictions.predictions.shape, predictions.label_ids.shape)

In [ ]:
import numpy as np
from datasets import list_metrics, load_metric

print(list_metrics())

metric = load_metric("ner","conllpp")
preds = np.argmax(predictions.predictions, axis=-1)
metric.compute(predictions=preds,references=predictions.label_ids)

In [ ]:
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions,reference=labels)

In [ ]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")

trainer = Trainer(model,
                  training_args,
                  train_dataset=tokenized_data["train"],
                  eval_dataset=tokenized_data["validation"],
                 data_collator = data_collator_token,
                 tokenizer=tokenizer,
                 compute_metrics=compute_metrics)
trainer.train()

In [44]:
from torch.utils.data import Dataset
class CustomDataset(Dataset):
    def __init__(self, features):
        self.features = features
    def __len__(self):
        return self.features.shape[0]
    def __getitem__(self, idx):
        item = self[idx]["tokens"]
        label = self[idx]["ner_tags"]
        return item, label